In [108]:
import pandas as pd
import numpy as np
import os
from urllib.request import urlretrieve
import partridge as ptg
import peartree as ptr
import plotly.express as px
import networkx as nx

In [109]:
CWD = os.getcwd()
DATA_PATH = os.path.join(CWD, 'data')
GTFS_PATH = os.path.join(DATA_PATH, 'gtfs')

In [110]:
df = pd.read_csv(os.path.join(DATA_PATH, "sources.csv"))
df

,mdb_source_id,data_type,entity_type,location.country_code,location.subdivision_name,location.municipality,provider,name,note,feed_contact_email,...,urls.license,location.bounding_box.minimum_latitude,location.bounding_box.maximum_latitude,location.bounding_box.minimum_longitude,location.bounding_box.maximum_longitude,location.bounding_box.extracted_on,status,features,redirect.id,redirect.comment
0,1,gtfs,NaN,US,Maine,Casco Bay,Casco Bay Lines,NaN,NaN,NaN,...,NaN,43.655373,43.716190,-70.248666,-70.110180,2022-02-22T19:51:22+00:00,inactive,NaN,NaN,NaN
1,2,gtfs,NaN,CA,Ontario,London,London Transit Commission,NaN,NaN,croy@londontransit.ca,...,https://www.londontransit.ca/open-data/ltcs-op...,42.905244,43.051188,-81.363110,-81.137591,2022-02-22T19:51:34+00:00,NaN,NaN,NaN,NaN
2,3,gtfs,NaN,CA,Ontario,Barrie,Barrie Transit,NaN,NaN,NaN,...,https://www.barrie.ca/services-payments/transp...,44.321804,44.420207,-79.740632,-79.610896,2022-03-01T22:43:25+00:00,NaN,NaN,NaN,NaN
3,4,gtfs,NaN,US,Ohio,Athens,Athens Public Transit,NaN,NaN,NaN,...,NaN,39.295665,39.399200,-82.141964,-82.027972,2022-03-23T14:43:11+00:00,deprecated,NaN,1973.0,NaN
4,5,gtfs,NaN,US,New York,Canton,St Lawrence County Public Transit,NaN,NaN,NaN,...,https://data.ny.gov/download/77gx-ii52/applica...,44.148476,44.979369,-75.756950,-74.611761,2022-03-14T20:02:31+00:00,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1972,1973,gtfs,NaN,US,Ohio,Athens,Athens Public Transit,NaN,NaN,info@athenspublictransit.org,...,NaN,39.223943,39.469958,-82.250627,-82.028000,2024-01-18T22:25:23+00:00,NaN,NaN,NaN,NaN
1973,1974,gtfs,NaN,US,California,Concord,Tri Delta Transit,NaN,NaN,511sfbaydeveloperresources@googlegroups.com,...,https://511.org/sites/default/files/pdfs/511_D...,37.895701,38.039856,-122.139478,-121.691400,2024-01-22T18:17:58+00:00,active,NaN,NaN,NaN
1974,1975,gtfs-rt,tu,US,California,Concord,Tri Delta Transit,NaN,NaN,NaN,...,https://511.org/sites/default/files/pdfs/511_D...,37.895701,38.039856,-122.139478,-121.691400,2024-01-22T18:17:58+00:00,NaN,NaN,NaN,NaN
1975,1976,gtfs-rt,vp,US,California,Concord,Tri Delta Transit,NaN,NaN,NaN,...,https://511.org/sites/default/files/pdfs/511_D...,37.895701,38.039856,-122.139478,-121.691400,2024-01-22T18:17:58+00:00,NaN,NaN,NaN,NaN


In [111]:
files_to_download = [20]
for file_name in files_to_download:
    path = os.path.join(GTFS_PATH, f'{file_name}.zip')
    if not os.path.exists(path):
        url = df.loc[file_name, "urls.direct_download"]
        urlretrieve(url, path)

In [112]:
example_feed = files_to_download[0]
in_path = os.path.join(GTFS_PATH, f'{example_feed}.zip')
graph_name = df.loc[example_feed]["location.municipality"].replace(" ", "-")
max_lats, max_longs, min_lats, min_longs = df.loc[example_feed][["location.bounding_box.maximum_latitude",
                                                            "location.bounding_box.maximum_longitude",
                                                            "location.bounding_box.minimum_latitude",
                                                            "location.bounding_box.minimum_longitude"]]

In [113]:
geo_shapes = ptg.load_geo_feed(in_path).shapes
feed = ptg.load_feed(in_path)

F:\anaconda\envs\DA_TrNet\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



In [114]:
feed.trips

,route_id,service_id,trip_id,trip_short_name,trip_headsign,direction_id,block_id,shape_id,bikes_allowed,wheelchair_accessible,trip_type,drt_max_travel_time,drt_avg_travel_time,drt_advance_book_min,drt_pickup_message,drt_drop_off_message,continuous_pickup_message,continuous_drop_off_message,tts_trip_headsign,tts_trip_short_name
0,13596,c_69622_b_80876_d_31,t_5652885_b_80876_tn_1,NaN,NaN,0,115049,p_1434000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13596,c_69622_b_80876_d_31,t_5652885_b_80876_tn_9,NaN,NaN,0,115049,p_1434000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13596,c_69622_b_80876_d_31,t_5652885_b_80876_tn_8,NaN,NaN,0,115049,p_1434000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13596,c_69622_b_80876_d_31,t_5652885_b_80876_tn_7,NaN,NaN,0,115049,p_1434000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13596,c_69622_b_80876_d_31,t_5652885_b_80876_tn_6,NaN,NaN,0,115049,p_1434000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,74351,c_69622_b_80876_d_31,t_5652892_b_80876_tn_1,NaN,Coquille,1,1359640,p_1433076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,74351,c_69622_b_80876_d_31,t_5652892_b_80876_tn_2,NaN,Coquille,1,1359640,p_1433076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,74351,c_69622_b_80876_d_31,t_5652893_b_80876_tn_2,NaN,Coquille,1,1359640,p_1433076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,74377,c_69622_b_80876_d_31,t_5652883_b_80876_tn_0,NaN,Fred Meyer via Eastside School,0,1295591,p_1425363,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
lats, longs, names = [], [], []
for row in list(geo_shapes.iterrows()):
    name, linestring = row[1]
    x, y = linestring.xy
    lats = np.append(lats, y)
    longs = np.append(longs, x)
    names = np.append(names, [name] * len(y))
    lats = np.append(lats, None)
    longs = np.append(longs, None)
    names = np.append(names, None)

In [116]:
fig = px.line_mapbox(lat=lats, lon=longs, hover_name=names, zoom=8)
fig.update_layout(mapbox_style="open-street-map", margin={"r":0,"t":0,"l":0,"b":0},
                  mapbox_center_lon=(max_longs + min_longs) / 2, mapbox_center_lat=(max_lats + min_lats) / 2)
fig.show()

In [117]:
feed.stops['stop_id'].unique()

array(['13451', '13455', '13459', '13461', '13462', '13464', '13468',
       '13469', '13470', '13476', '13480', '13485', '13488', '13490',
       '13492', '13493', '13496', '13498', '13504', '13510', '13513',
       '13515', '13517', '20493', '2388673', '2388674', '2388675',
       '2388676', '2484482', '2537768', '2537770', '2537772', '2537773',
       '2537774', '2537775', '2537776', '2537778', '2537789', '2617901',
       '2617903', '2617905', '2617958', '2617960', '2617961', '2617962',
       '2617963', '2617965', '2617966', '2617967', '26294', '4147145',
       '4147146', '4147147', '4147148', '4147149', '4147150', '4147151',
       '4147152', '4147153', '4147154', '4147155', '4147156', '4147157',
       '4147158', '4147159', '4147600', '4147601', '4147602', '4147603',
       '4147604', '4147697', '4209068', '4209716', '4209719', '4209725',
       '4209727', '4212210', '4212223', '783786'], dtype=object)

In [121]:
start = 7*60*60
end = 10*60*60
g = ptr.load_feed_as_graph(feed, start, end, name=graph_name)
# ptr.generate_plot(g)

In [122]:
nx.number_of_nodes(g)

69

In [137]:
g.edges(data=True)

OutMultiEdgeDataView([('Coos-Bay_13451', 'Coos-Bay_13469', {'length': 60.0, 'mode': 'transit'}), ('Coos-Bay_13455', 'Coos-Bay_13480', {'length': 120.0, 'mode': 'transit'}), ('Coos-Bay_13455', 'Coos-Bay_13498', {'length': 420.0, 'mode': 'transit'}), ('Coos-Bay_13455', 'Coos-Bay_2388673', {'length': 540.0, 'mode': 'transit'}), ('Coos-Bay_13455', 'Coos-Bay_13513', {'length': 900.0, 'mode': 'transit'}), ('Coos-Bay_13459', 'Coos-Bay_13461', {'length': 125.0, 'mode': 'transit'}), ('Coos-Bay_13459', 'Coos-Bay_4212210', {'length': 493.0, 'mode': 'transit'}), ('Coos-Bay_13461', 'Coos-Bay_13498', {'length': 360.0, 'mode': 'transit'}), ('Coos-Bay_13462', 'Coos-Bay_13464', {'length': 300.0, 'mode': 'transit'}), ('Coos-Bay_13464', 'Coos-Bay_4212223', {'length': 75.0, 'mode': 'transit'}), ('Coos-Bay_13468', 'Coos-Bay_2537773', {'length': 60.0, 'mode': 'transit'}), ('Coos-Bay_13468', 'Coos-Bay_13455', {'length': 900.0, 'mode': 'transit'}), ('Coos-Bay_13469', 'Coos-Bay_13470', {'length': 60.0, 'mode':

In [130]:
feed.stops.loc[feed.stops['stop_id'] == '2388673']

,stop_id,stop_code,platform_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,position,direction,wheelchair_boarding,tts_stop_name
24,2388673,NaN,NaN,4th & Central,NaN,43.36773,-124.216075,8856,NaN,0,NaN,America/Los_Angeles,NaN,NaN,0,NaN


In [135]:
feed.stops.loc[feed.stops['stop_name'] == "Winchester Bay"]

,stop_id,stop_code,platform_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,position,direction,wheelchair_boarding,tts_stop_name
42,2617960,NaN,NaN,Winchester Bay,NaN,43.673815,-124.176463,8853,NaN,0,NaN,America/Los_Angeles,NaN,NaN,0,NaN
48,2617967,NaN,NaN,Winchester Bay,NaN,43.674121,-124.176303,8853,NaN,0,NaN,America/Los_Angeles,NaN,NaN,0,NaN
